In [5]:
import numpy as np
import sympy as sp
from sympy import Point2D

from IPython.display import display
from IPython.display import Latex

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

def latex(m):
    return sp.latex(sp.Matrix(m), mat_str='pmatrix').replace(r'\left[', '').replace(r'\right]', '')

def disp(text):
    # display(Latex(str))
    if isinstance(text, str):
        # Разбиваем по строкам и отображаем каждую отдельно
        lines = text.strip().split('\n')
        for line in lines:
            if line.strip():  # Пропускаем пустые строки
                display(Latex(line.strip()))
    else:
        display(Latex(str(text)))

A, B, C, D = 2, 3, 4, 5

In [6]:
points = list()
colors = list()
for x in range(-4, 5):
    for y in range(-4, 5):
        points.append([x, y])
        colors.append(plt.cm.hsv((y + 13*x)/15 % 1))  # Example color based on y-coordinate
points = np.array(points).T

def set_auto_limits(points, margin=2):
    # points shape: (2, N) или (N, 2)
    pts = points if points.shape[0] != 2 else points.T
    x_min, x_max = np.min(pts[:,0]), np.max(pts[:,0])
    y_min, y_max = np.min(pts[:,1]), np.max(pts[:,1])
    plt.xlim(x_min - margin, x_max + margin)
    plt.ylim(y_min - margin, y_max + margin)

plt.scatter(*points, c=colors)
# plt.scatter(*points)
plt.gca().set_aspect('equal')   # равные масштабы по осям
set_auto_limits(points, margin=2)
plt.grid()
plt.show()

disp(latex(points))

In [7]:
# ...existing code...

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

def rotation_matrix(theta):
    return np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta),  np.cos(theta)]
    ])

fig, ax = plt.subplots()
# points должен быть shape (N, 2), colors длины N
if points.shape[0] == 2:  # points сейчас shape (2, N)
    points_plot = points.T
else:
    points_plot = points

scat = ax.scatter(points_plot[:,0], points_plot[:,1], c=colors)
ax.set_aspect('equal')
ax.set_xlim(-8, 8)
ax.set_ylim(-8, 8)
ax.grid()

frames = 72
final_angle = np.pi / 4

def animate(frame):
    theta = final_angle * frame / (frames - 1)
    mat = rotation_matrix(theta)
    transformed = (mat @ points).T if points.shape[0] == 2 else points @ mat.T
    scat.set_offsets(transformed)
    return scat,

ani = FuncAnimation(fig, animate, frames=frames, interval=50, blit=True)

ani.save("rotation.gif", writer=PillowWriter(fps=20))

plt.close(fig)

from IPython.display import Image
Image(filename="rotation.gif")

# ...existing code...

In [8]:
def show_points_before_after_subplots(points_before, points_after, lim = 7, colors=None):
    # points_before и points_after: shape (N, 2) или (2, N)
    def to_plot_shape(arr):
        return arr.T if arr.shape[0] == 2 else arr

    pts_before = to_plot_shape(points_before)
    pts_after = to_plot_shape(points_after)

    n_pts = pts_before.shape[0]
    if colors is None:
        colors = ['red'] * n_pts
    if len(colors) < n_pts:
        colors = (colors * (n_pts // len(colors) + 1))[:n_pts]


    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    # До преобразования
    axs[0].scatter(pts_before[:,0], pts_before[:,1], c=colors)
    axs[0].set_title("До преобразования")
    axs[0].set_xlim(-lim, lim)
    axs[0].set_ylim(-lim, lim)
    axs[0].set_aspect('equal')
    axs[0].grid()

    # После преобразования
    axs[1].scatter(pts_after[:,0], pts_after[:,1], c=colors)
    axs[1].set_title("После преобразования")
    axs[1].set_xlim(-lim, lim)
    axs[1].set_ylim(-lim, lim)
    axs[1].set_aspect('equal')
    axs[1].grid()

    # plt.show()
    
    return fig, axs

## Отражение (симметрию) плоскости относительно прямой $y = ax$

In [20]:
# Good
points1 = np.array([
    [4, 2],
    [6, -2],
    [0, -6],
    [-4, -4],
    # [-8, 0],
    [-4, 6],
    [0, 4]
]).T
N = points1.shape[1] if points1.shape[0] == 2 else points1.shape[0]
colors1 = [plt.cm.hsv(i / N) for i in range(N)]

S = np.array([
    [1, 0],
    [0, -1]
])

P = np.array([
    [1, -A],
    [A, 1]
])

A1 = P @ S @ np.linalg.inv(P)

theta = np.arctan(A)
cos_theta = np.cos(2*theta)
sin_theta = np.sin(2*theta)
matrix1 = np.array([
    [cos_theta, sin_theta],
    [sin_theta, -cos_theta]
])

matrix2 = 1/(1 + A**2) * np.array([
    [1-A**2, 2*A],
    [2*A, A**2-1]
])

matrix = A1
new_points1 = matrix @ points1

fig, axs = show_points_before_after_subplots(points1, new_points1, colors=colors1, lim=9)
xs = np.linspace(-4, 4, 100)
ys = A * xs
axs[1].plot(xs, ys, 'b--', label=f'y={A}x')
axs[0].plot(xs, ys, 'b--', label=f'y={A}x')
plt.legend()
plt.show()

disp(latex(points))


disp(f"""
    В стандартном базисе отражение нам задавать не удобно, поэтому зададим её в новом базисе:
    В этом новом базисе отражение становится очень простым:\\\\
    Координата вдоль прямой (по x) не меняется.\\\\
    Координата перпендикулярно прямой (по y) меняет знак.\\\\
     
    Тогда матрица преобразования в новом базисе будет выглядить так:
    $$S = {latex(S)}$$
    Задача — вернуться в стандартный базис и найти там матрицу A, но перед этим зададим наш новый базис:\\\\
    Вектор вдоль прямой x: направляющий вектор прямой — $(1, a)$.\\\\
    Вектор, перпендикулярный прямой(y) - это вектор, перпендикулярный (1, a) -> (-a, 1). Запишем новый базис по столбцам - это и будет наша матрица перехода:
     
    $$P = {latex(P)}$$
    Находим матрицу пробразования A в стандартном базисе по формуле $A = P S P^{-1}$:
    $$A = {latex(A1)}$$

    """)
disp(f"{latex(A1)}{latex(points1)} = {latex(A1 @ points1)}")

In [16]:
theta = np.arctan(A)
cos_theta = np.cos(2*theta)
sin_theta = np.sin(2*theta)
matrix = np.array([
    [cos_theta, sin_theta],
    [sin_theta, -cos_theta]
])
new_points = matrix @ points

fig, axs = show_points_before_after_subplots(points, new_points, colors=colors)
xs = np.linspace(-8, 8, 100)
ys = A * xs
axs[1].plot(xs, ys, 'b--', label=f'y={A}x')
axs[0].plot(xs, ys, 'b--', label=f'y={A}x')
plt.legend()
plt.show()

# disp(latex(points))

## Ортогональное отображение всей плоскости в прямую $y = bx$

Направляющий вектор: $\bar{v} = \begin{bmatrix} 1 \\ B \end{bmatrix}$

$$ l = \frac{\left(p \cdot v\right)}{\left(v \cdot v\right)} $$
$$ v' = \frac{\left(p \cdot v\right)}{\left(v \cdot v\right)} v 
      = \frac{\left(p \cdot v\right)}{\left(v \cdot v\right)} \begin{bmatrix} 1 \\ B \end{bmatrix}
$$

Для Базиса B: $\begin{bmatrix} 1 & -B \\ B & 1 \end{bmatrix}$

In [17]:
_points = points.T[::].T
_colors = colors[::]

v = np.array([1, B])
new_points = np.zeros_like(_points)
for i, point in enumerate(_points.T):
    proj_length = (point @ v) / (v @ v)
    new_points[:, i] = proj_length * v
    # print(f"Point: {point}, Projected: {new_point}")

fig, axs = show_points_before_after_subplots(_points, new_points, colors=_colors)
xs = np.linspace(-2, 2, 100)
ys = B * xs
axs[0].plot(xs, ys, 'b--', label=f'y={B}x')
axs[1].plot(xs, ys, 'b--', label=f'y={B}x')
plt.legend()
plt.show()

In [21]:
# _every = 10
# _points = points.T[::_every].T
# _colors = colors[::_every]

_points = points1
_colors = colors1

EtoB = np.array([[1, -B], 
                 [B, 1]])
BtoE = np.linalg.inv(EtoB)

zeroy = np.array([[1, 0],
                  [0, 0]])
new_points = BtoE @ zeroy @ EtoB @ _points
new_points = EtoB @ zeroy @ BtoE @ _points

fig, axs = show_points_before_after_subplots(_points, new_points, colors=_colors)
xs = np.linspace(-2, 2, 100)
ys = B * xs
axs[0].plot(xs, ys, 'b--', label=f'y={B}x')
axs[1].plot(xs, ys, 'b--', label=f'y={B}x')
plt.legend()
plt.show()

disp(latex(BtoE @ zeroy @ EtoB))

## Поворот плоскости на $10c$ градусов против часовой стрелки.